In [1]:
%load_ext autoreload
%autoreload 2

from utils.loading import read_saved_file

In [2]:
sequence_df = read_saved_file(type_to_read='Sequence')
taxonomy_df = read_saved_file(type_to_read='Taxonomy')

In [28]:
joined_df = sequence_df.merge(taxonomy_df.drop('file_num', axis=1), on = ['reference'], how='left')

In [35]:
working_df = joined_df.dropna(axis=0, subset=['kingdom'])

In [39]:
working_df.phylum.value_counts()

PROTEOBACTERIA    23510
FIRMICUTES        22212
BACTEROIDETES      8356
ACTINOBACTERIA     5315
CHLOROFLEXI        2530
                  ...  
NANOARCHAEOTA         4
DICTYOGLOMI           3
CD12                  3
GOUTA4                3
WS6                   2
Name: phylum, Length: 68, dtype: int64